# Prudhviraj 
### Google Data Analyst Assignment 
### Ford Go Bike Data Analysis

In [280]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [281]:
dataframe1 = pd.read_csv("2017-fordgobike-tripdata.csv.csv", sep=',', header=0, low_memory=False , encoding='latin-1')
dataframe2 = pd.read_csv("201801-fordgobike-tripdata.csv", sep=',', header=0, low_memory=False , encoding='latin-1')
dataframe3 = pd.read_csv("201802-fordgobike-tripdata.csv", sep=',', header=0, low_memory=False , encoding='latin-1')
dataframe4 = pd.read_csv("201803-fordgobike-tripdata.csv", sep=',', header=0, low_memory=False , encoding='latin-1')

In [282]:
dataframe = pd.concat([dataframe1, dataframe2, dataframe3, dataframe4], join='inner', ignore_index=True)
dataframe.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender
0,80110,2017-12-31 16:57:39.6540,2018-01-01 15:12:50.2450,74,Laguna St at Hayes St,37.776435,-122.426244,43,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96,Customer,1987.0,Male
1,78800,2017-12-31 15:56:34.8420,2018-01-01 13:49:55.6170,284,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,96,Dolores St at 15th St,37.766210,-122.426614,88,Customer,1965.0,Female
2,45768,2017-12-31 22:45:48.4110,2018-01-01 11:28:36.8830,245,Downtown Berkeley BART,37.870348,-122.267764,245,Downtown Berkeley BART,37.870348,-122.267764,1094,Customer,NaN,NaN
3,62172,2017-12-31 17:31:10.6360,2018-01-01 10:47:23.5310,60,8th St at Ringold St,37.774520,-122.409449,5,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,2831,Customer,NaN,NaN
4,43603,2017-12-31 14:23:14.0010,2018-01-01 02:29:57.5710,239,Bancroft Way at Telegraph Ave,37.868813,-122.258764,247,Fulton St at Bancroft Way,37.867789,-122.265896,3167,Subscriber,1997.0,Female


In [283]:
dataframe.info() # there is no null data in the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 832602 entries, 0 to 832601
Data columns (total 15 columns):
duration_sec               832602 non-null int64
start_time                 832602 non-null object
end_time                   832602 non-null object
start_station_id           832602 non-null int64
start_station_name         832602 non-null object
start_station_latitude     832602 non-null float64
start_station_longitude    832602 non-null float64
end_station_id             832602 non-null int64
end_station_name           832602 non-null object
end_station_latitude       832602 non-null float64
end_station_longitude      832602 non-null float64
bike_id                    832602 non-null int64
user_type                  832602 non-null object
member_birth_year          741095 non-null float64
member_gender              741286 non-null object
dtypes: float64(5), int64(4), object(6)
memory usage: 95.3+ MB


## Average Trip duration in seconds

In [284]:
dataframe['duration_sec'].mean()

1014.2486866474018

## In Minutes Divide the above result with 60 

In [285]:
dataframe['duration_sec'].mean()/60

16.904144777456697

## Standard Deviation in seconds

In [286]:
dataframe['duration_sec'].std()

3149.681666336765

## In Minutes divide the above result with 60 

In [287]:
dataframe['duration_sec'].std()/60

52.494694438946084

the other way to get average and standard deviation is by using .describe()

## Finding the busiest dates:

####  Upon on carefully reviewing the ford go bike plan details at "https://www.fordgobike.com/pricing" a trip can last upto a month

In [288]:
# first lets make a dummy dataframe of just the start time and the end time and call it dummy

dummy  = dataframe[['start_time', 'end_time']]

dummy.head()

,start_time,end_time
0,2017-12-31 16:57:39.6540,2018-01-01 15:12:50.2450
1,2017-12-31 15:56:34.8420,2018-01-01 13:49:55.6170
2,2017-12-31 22:45:48.4110,2018-01-01 11:28:36.8830
3,2017-12-31 17:31:10.6360,2018-01-01 10:47:23.5310
4,2017-12-31 14:23:14.0010,2018-01-01 02:29:57.5710


In [289]:
# the dummy dataframe consists of data and time. for us to analyze the busy dates we need to get rid of the time 
# lets use python split function to seperate data and time 

#let us write a helper function to do our task and call it when necessary

def date(string):
    return (string.split(' ')[0])

In [290]:
dummy['start_time'] = dummy['start_time'].apply(date)
dummy['end_time'] = dummy['end_time'].apply(date)

In [291]:
dummy.head()

,start_time,end_time
0,2017-12-31,2018-01-01
1,2017-12-31,2018-01-01
2,2017-12-31,2018-01-01
3,2017-12-31,2018-01-01
4,2017-12-31,2018-01-01


In [292]:
count = dummy.start_time.value_counts()
# count.head()
# from here we can say that the busy day is march 29 2018 but we cannot come to the conclusion taking jus the starttime
# what if the bike is rented for a month or 3 days using monthly or 3 day pass.
#here we need to consider the mutiple rental records.
#for this the startdate and end date varies and start date is always less than the end date.

multiple = dummy.loc[(dummy['start_time']< dummy['end_time'])]
#gives only the values for which multiple dates is true.

In [293]:
# now let us loop through the multiple data frame and append the number of days between the start and end to the
# date in the count dataframe.
#source 

from datetime import *

for index, row in multiple.iterrows():
    start = datetime.strptime(row['start_time'], "%Y-%m-%d")
    end = datetime.strptime(row['end_time'], "%Y-%m-%d")
    delta = end - start
    for i in range(1, delta.days+1):
        date = str((start+timedelta(i)).date())
        if date in count:
            count[date] +=1
        else:
            count[date] = 1   

In [294]:
daycount.head()

2018-03-29    5445
2018-03-28    5372
2018-02-08    5232
2018-03-27    5192
2018-02-06    5175
Name: start_time, dtype: int64

#### The busiest day is on march 29 2018

### Now let's find the busiest time

In [295]:
dummytime.head()

,duration_sec,start_time,end_time
0,80110,2017-12-31,2018-01-01
1,78800,2017-12-31,2018-01-01
2,45768,2017-12-31,2018-01-01
3,62172,2017-12-31,2018-01-01
4,43603,2017-12-31,2018-01-01


In [296]:
#We need to calcuate the busiest time of the day, that is at what particular hour in 24 hour span.

dummy2  = dataframe[['duration_sec','start_time', 'end_time']]

#initialise the time set, we are using set here cause we don't want any redundant time values.
timecount = {}

for index,row in dummy2.iterrows():
    time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S.%f").hour
    number_of_hours = int(row['duration_sec']/3600) #give in seconds, to convert into hours 6/0X60
    for i in range(number_of_hours +1):
        if (time+i)%24 in timecount:
            timecount[(time+i)%24] +=1
        else:
            timecount[(time+i)%24] =1

In [297]:
print("The busiest time in a day @",'%d%s'%((max(timecount,key=timecount.get))-12,"PM") if max(timecount,key=timecount.get)>12 else '%d%s'%((max(timecount,key=timecount.get),"AM")))

The busiest time in a day @ 5PM


### Most common Start and End Stations 

In [298]:
#Lets explore the most common start and end stations of bike rental
start_station = dataframe.start_station_name.value_counts()
print("The Most common start Station where the bikes are rented is @:", start_station.head(1))
end_station = dataframe.end_station_name.value_counts()
print("The most common end Station to dock the bikes is @:", end_station.head(1))

The Most common start Station where the bikes are rented is @: San Francisco Ferry Building (Harry Bridges Plaza)    22357
Name: start_station_name, dtype: int64
The most common end Station to dock the bikes is @: San Francisco Caltrain (Townsend St at 4th St)    26787
Name: end_station_name, dtype: int64


### Common Start Station is San Francisco Ferry Building (Harry Bridges Plaza) 
### Common End Station is San Francisco Caltrain (Townsend St at 4th St)

## Exploring the User Space

### User population description

In [299]:
# let us first generate the user age by using the following helper function.
# the following helper function will take the user year of birth and returns the current age.

def userage(yob):
    if yob == 'Nan':
        return 'Nan'
    else:
        return 2018-yob

In [300]:
dataframe['age'] = dataframe.apply(lambda x: userage(x['member_birth_year']), axis=1)
dataframe.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,age
0,80110,2017-12-31 16:57:39.6540,2018-01-01 15:12:50.2450,74,Laguna St at Hayes St,37.776435,-122.426244,43,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96,Customer,1987.0,Male,31.0
1,78800,2017-12-31 15:56:34.8420,2018-01-01 13:49:55.6170,284,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,96,Dolores St at 15th St,37.766210,-122.426614,88,Customer,1965.0,Female,53.0
2,45768,2017-12-31 22:45:48.4110,2018-01-01 11:28:36.8830,245,Downtown Berkeley BART,37.870348,-122.267764,245,Downtown Berkeley BART,37.870348,-122.267764,1094,Customer,NaN,NaN,NaN
3,62172,2017-12-31 17:31:10.6360,2018-01-01 10:47:23.5310,60,8th St at Ringold St,37.774520,-122.409449,5,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,2831,Customer,NaN,NaN,NaN
4,43603,2017-12-31 14:23:14.0010,2018-01-01 02:29:57.5710,239,Bancroft Way at Telegraph Ave,37.868813,-122.258764,247,Fulton St at Bancroft Way,37.867789,-122.265896,3167,Subscriber,1997.0,Female,21.0


In [301]:
agecount = dataframe.age.value_counts()
agecount.head()

30.0    39849
33.0    38512
31.0    36713
34.0    33850
36.0    33322
Name: age, dtype: int64

### the people who are in their 30's use the service more than others

#### lets get deeper into the analysis.

In [302]:
dataframe['age'].describe()

count    741095.000000
mean         37.172067
std          10.623434
min          18.000000
25%          30.000000
50%          35.000000
75%          43.000000
max         132.000000
Name: age, dtype: float64

In [303]:
# it can be see from the above description that the Max age of the person has exceed 100
# may be he/she should be enterd into guniess book ....haha!!
# Now we have to remove such outliers and wrong data
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 832602 entries, 0 to 832601
Data columns (total 16 columns):
duration_sec               832602 non-null int64
start_time                 832602 non-null object
end_time                   832602 non-null object
start_station_id           832602 non-null int64
start_station_name         832602 non-null object
start_station_latitude     832602 non-null float64
start_station_longitude    832602 non-null float64
end_station_id             832602 non-null int64
end_station_name           832602 non-null object
end_station_latitude       832602 non-null float64
end_station_longitude      832602 non-null float64
bike_id                    832602 non-null int64
user_type                  832602 non-null object
member_birth_year          741095 non-null float64
member_gender              741286 non-null object
age                        741095 non-null float64
dtypes: float64(6), int64(4), object(6)
memory usage: 101.6+ MB


In [304]:
dataframe_age = dataframe

In [305]:
dataframe_age = dataframe_age[np.isfinite(dataframe_age['age'])] # all the rows with Nan values in Age are removed
dataframe_age = dataframe_age[dataframe_age.age < 101]

In [306]:
dataframe_age['age'].describe()

count    740366.000000
mean         37.095424
std          10.343104
min          18.000000
25%          30.000000
50%          35.000000
75%          43.000000
max          94.000000
Name: age, dtype: float64

In [309]:
agecount2 = dataframe_age.age.value_counts()
agecount2.head()

30.0    39849
33.0    38512
31.0    36713
34.0    33850
36.0    33322
Name: age, dtype: int64

In [310]:
# Dataframe for the age group and the number of rides.
agecount2 = pd.DataFrame({'age':agecount2.index,'Number of rides':agecount2.values})
agecount2 

,Number of rides,age
0,39849,30.0
1,38512,33.0
2,36713,31.0
3,33850,34.0
4,33322,36.0
5,33286,29.0
6,32536,32.0
7,31261,35.0
8,28677,28.0
9,28376,27.0


### The Service is mostly used by  people in  age range between 30 -36

### will come up with ML models soon on this Keep checking. 

##### Prudhviraj Tirumanisetti